In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
dataset_names_internal = ["indices_16.json", "bands+indices_6_unlim.json", "bands+indices_11_unlim.json"]
# dataset_names_internal = ["fest_16.json"]
dataset_names = [16, 600, 1100]
dataset = [None]*len(dataset_names_internal)

for i in range(len(dataset_names_internal)):
    file_to_read = open(rf"models_results_default\{dataset_names_internal[i]}", "rb")
    dataset[i] = json.load(file_to_read)

In [3]:
def dataset_to_df(dataset, dataset_name, loc_id_offset):
    df_dic = { "loc_id": [], "config": [], "key": [], "iou (train)": [], "pr_auc (train)": [], "iou (test)": [], "pr_auc (test)": [] }

    counter = loc_id_offset
    for key in dataset.keys():
        results = dataset[key]
        dic = { "iou (train)": [], "pr_auc (train)": [], "iou (test)": [], "pr_auc (test)": [] }
        for ev in results.keys():
            for k in results[ev].keys():
                if k in dic.keys():
                    dic[k].append(results[ev][k])

        for k in dic.keys():
            dic[k] = np.max(np.array(dic[k]))

        df_dic["loc_id"].append(counter)
        df_dic["config"].append(key)
        df_dic["key"].append(dataset_name)
        for k in dic.keys():
            df_dic[k].append(dic[k])

        counter += 1

    return pd.DataFrame.from_dict(df_dic)

In [4]:
dfs = []

dfs.append(dataset_to_df(dataset[0], dataset_names[0], 0))
for i in range(1, len(dataset_names_internal)):
    dfs.append(dataset_to_df(dataset[i], dataset_names[i], 1))

df = pd.concat(dfs)

In [5]:
df.sort_values(["loc_id", "key"], inplace=True)
# df.sort_values(["pr_auc (train)"], inplace=True, ascending=False)
# df.sort_values(["iou (test)", "pr_auc (test)"], inplace=True, ascending=False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df.head(1000)

,loc_id,config,key,iou (train),pr_auc (train),iou (test),pr_auc (test)
0,0,BANDS,16,0.623152,0.843376,0.560511,0.779941
1,1,NORMP,16,0.604207,0.831986,0.550752,0.746805
0,1,NORMP,600,0.642089,0.861601,0.558505,0.767833
0,1,NORMP,1100,0.628680,0.856286,0.554986,0.764245
2,2,NORMP4,16,0.604066,0.832035,0.555028,0.755693
1,2,NORMP4,600,0.636444,0.857313,0.555431,0.768940
1,2,NORMP4,1100,0.645488,0.863373,0.553683,0.755037
3,3,DIST2,16,0.609635,0.836987,0.556401,0.777004
2,3,DIST2,600,0.636885,0.855426,0.546077,0.754083
2,3,DIST2,1100,0.633764,0.852968,0.557645,0.766239


In [6]:
pd.DataFrame.to_csv(df, "max.csv")і